<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/01_Sentinel_2_MSI_1C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract seasonal and annual NDVI from Sentinel 2

See [here](https://developers.google.com/earth-engine/guides/python_install#syntax) for differences between Javascript and Python syntax

Main [tutorial](https://courses.spatialthoughts.com/end-to-end-gee.html#module-6-google-earth-engine-python-api) here

## Import gee and authenticate

Expiry period: the granted permissions will expire in a week, after which you'll need to call ee.Authenticate() again.

In [ ]:
# This doesn't really work, it's only valid for the current runtime...
import os

# If credentials file doesn't exist, authenticate and store credentials
# Else if credentials file does exist, use stored credentials and initialise
if not os.path.exists(os.path.expanduser("~/.config/earthengine/credentials")):
  import ee
  ee.Authenticate()
  ee.Initialize()
else:
  import ee
  ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
# aoi_name = "MKR_NS_buff_5km"

aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_PACE")
aoi_name = "MKR_PACE"

# GEE layer ID
layer_name = "RS_001"

# Image reducer (options: mean, median, min, max, stdDev, sum, product)
img_col_reducer = "mean"

# Date parameters (for sentinel there aren't 10 years of data available)
start_year = 2016
end_year = 2022

# Range doesn't include the stop value
year_list = ee.List(list(range(start_year, end_year+1)))

# Season parameters (months)
rain_start = 3
rain_end = 5 
dry_start = 7
dry_end = 10

## Import Sentinel 2 image collection

Dataset starts in June 2015
Clouds can be mostly removed by using [COPERNICUS/S2_CLOUD_PROBABILITY](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY). See [this tutorial](https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless) explaining how to apply the cloud mask.



---


Sentinel-2 MSI: MultiSpectral Instrument, Level-1C
```ee.ImageCollection("COPERNICUS/S2")```



---


Harmonized Sentinel-2 MSI: MultiSpectral Instrument, Level-2A
```ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")```



---


Sentinel-2 MSI: MultiSpectral Instrument, Level-2A
```ee.ImageCollection("COPERNICUS/S2_SR")```

In [ ]:
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

## NDVI processing

### Helper functions

In [ ]:
def maskS2clouds(image):
  qa = image.select('QA60')
  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(10000)


def addNDVI(image):
  imgb = image.select('B.*')
  ndvi = imgb.normalizedDifference(['B8','B4']).rename('NDVI')
  return image.addBands(ndvi)

reducer_list = ee.Reducer.mean() \
.combine(reducer2 = ee.Reducer.median(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.min(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.max(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.stdDev(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.sum(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.product(), sharedInputs=True)

### Annual NDVI

In [ ]:
def annual_NDVI(year_date):
  start = ee.Date.fromYMD(year_date, 1, 1)
  end = ee.Date.fromYMD(year_date, 12, 31)
  date_range = ee.DateRange(start, end)
  name = start.format('YYYY_MM').cat('_to_').cat(end.format('YYYY_MM'))
  return ee.ImageCollection("COPERNICUS/S2") \
        .filterDate(date_range) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .map(addNDVI) \
        .select('NDVI') \
        .reduce(reducer = reducer_list) \
        .clip(aoi) \
        .set({'name': name})

annual_ndvi = year_list.map(annual_NDVI)

## Check an element of list
year_mosaic  = ee.Image(annual_ndvi.get(2))
label = ee.String(year_mosaic.get('name')).getInfo()
print(label)
print(year_mosaic.getInfo())

## Check a reducer band
band_select = ".*" + img_col_reducer
print(band_select)
print(year_mosaic.select(band_select).getInfo())

### Seasonal NDVI

In [ ]:
def annual_seasonal_NDVI(year_date, season_start, season_end):
  start = ee.Date.fromYMD(year_date, season_start, 1)
  end = ee.Date.fromYMD(year_date, season_end, 30)
  date_range = ee.DateRange(start, end)
  season_label = "dry" if season_start == 7 else "wet" 
  return ee.ImageCollection("COPERNICUS/S2") \
        .filterDate(date_range) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .map(addNDVI) \
        .select('NDVI') \
        .reduce(reducer = reducer_list) \
        .clip(aoi) \
        .set({'season': season_label,
              'year': year_date})

def map_seasonal_dry(year):
  return annual_seasonal_NDVI(year, dry_start, dry_end)

annual_dry = year_list.map(map_seasonal_dry)

def map_seasonal_rain(year):
  return annual_seasonal_NDVI(year, rain_start, rain_end)

annual_rain = year_list.map(map_seasonal_rain)

## Check an element of list
year_mosaic = ee.Image(annual_rain.get(0))
label = ee.String(year_mosaic.get('season')).getInfo() + str(ee.String(year_mosaic.get('year')).getInfo())
print(label)
print(year_mosaic.getInfo())
print(year_mosaic.bandNames().getInfo())

## Export data - create task

`filenamePrefix` should be in format: place_layer_timeperiod

In [ ]:
# Annual images
for i in range(ee.List.length(annual_ndvi).getInfo()):
  band_select = ".*" + img_col_reducer
  output_img =  ee.Image(annual_ndvi.get(i))
  output_img = output_img.select(band_select)
  output_name = f"{layer_name}_{img_col_reducer}_{aoi_name}_{ee.String(output_img.get('name')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 30,
                                     maxPixels = 10e12
                                     )
  task.start()
  print("STARTED TASK ", i+1)

In [ ]:
# Seasonal images - DRY
for i in range(ee.List.length(annual_dry).getInfo()):
  band_select = ".*" + img_col_reducer
  output_img =  ee.Image(annual_dry.get(i))
  output_img = output_img.select(band_select)
  output_name = f"{layer_name}_{img_col_reducer}_{aoi_name}_{ee.String(output_img.get('year')).getInfo()}_{ee.String(output_img.get('season')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 30,
                                     maxPixels = 10e12
                                     )
  task.start()
  print("STARTED TASK ", "DRY ", i+1)

In [ ]:
# Seasonal images - RAIN
for i in range(ee.List.length(annual_rain).getInfo()):
  band_select = ".*" + img_col_reducer
  output_img =  ee.Image(annual_rain.get(i))
  output_img = output_img.select(band_select)
  output_name = f"{layer_name}_{img_col_reducer}_{aoi_name}_{ee.String(output_img.get('year')).getInfo()}_{ee.String(output_img.get('season')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 30,
                                     maxPixels = 10e12
                                     )
  task.start()
  print("STARTED TASK ", "RAIN ", i+1)

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:ee.List.length(year_list).getInfo()]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

## Mount and unmount Google Drive


In [ ]:
# Mount
#from google.colab import drive

#ROOT = '/content/drive' 
#drive.mount(ROOT, force_remount=True) 


In [ ]:
# Unmount
#drive.flush_and_unmount() 